In [1]:
import customtkinter
import tkinter as tk
import openpyxl
import pandas as pd
import xlstools

class App(customtkinter.CTk):
    def __init__(self):
        super().__init__()

        self.contraints_excel_filepath = '//FS-201-Radioterapia.intecnus.org.ar/fisicos/8 - Físicos Médicos/Roy/INTECNUS-PreScript/protocols/Protocolo de constraints.xlsx'  # Update this path

        self.presc_frame = customtkinter.CTkFrame(self)
        self.presc_frame.pack(pady=20, padx=20, fill="both", expand=True)

        # Prescription template selection
        self.presc_templates = xlstools.get_cell_content(file_path=self.contraints_excel_filepath, cell_coordinate='B2', sheet_name=None)[2:]

        self.presc_menu, self.chosen_presc_template = self.create_dropdown_menu(
            self.presc_frame, 'Template de Prescripción', self.presc_templates, 
            row=6, column=0, callback=self.update_presc_template
        )
        self.presc_template = self.chosen_presc_template.get()
        self.entry_widgets = [self.chosen_presc_template]

        # Image protocol template selection

        self.images_menu, self.chosen_images_template = self.create_dropdown_menu(
            self.presc_frame, 'Protocolo de Imágenes', self.images_templates, 
            row=7, column=0, callback=self.update_images_template
        )
        self.entry_widgets.append(self.chosen_images_template)
        self.images_template = self.chosen_images_template.get()

    def get_images_template(self):
        images_template = xlstools.cell_data_importer(
            openpyxl.load_workbook(self.contraints_excel_filepath, read_only=True)['General'],
            (3,'E'), 
            (21,'E')
        )
        images_template = [item for sublist in images_template for item in sublist if item != 'None']
        return images_template

    def update_presc_template(self, new_template):
        self.presc_template = new_template
        print(f"Updated presc_template to: {self.presc_template}")
        self.update_images_menu()
        self.update_presc_data()

    def update_images_menu(self):
        self._create_images_template_based_on_presc_template()

        default_images_template = self.images_template_per_presc_dict.set(self.chosen_presc_template)

        self.images_menu.set(default_images_template)
        self.chosen_images_template.set(default_images_template)
        self.images_template = default_images_template
        print(f"Updated images_template to: {self.images_template}")

    def update_images_template(self, new_template):
        self.images_template = new_template
        print(f"Updated images_template to: {self.images_template}")

    def update_presc_data(self):
        actual_presc_data, _ = xlstools.raw_importer(self.contraints_excel_filepath, self.presc_template)
        actual_presc_data = actual_presc_data[:, :3]
        self.actual_presc_data_df = pd.DataFrame(actual_presc_data[1:], columns=actual_presc_data[0])

    def _create_images_template_based_on_presc_template(self):

        default_templates = xlstools.get_cell_content(file_path=self.contraints_excel_filepath, cell_coordinate='G5', sheet_name=None)[2:]
        
        if len(self.presc_templates) != len(default_templates):
            raise ValueError("ERROR: self.presc_templates and default_images_templates must have equal lengths")
        try:
            self.images_template_per_presc_dict = dict(zip(self.presc_templates, default_templates))
        except ValueError:
            return None  # or you could raise an exception if you prefer


    def create_dropdown_menu(self, frame, text, options, row, column, width=250, padx=20, pady=10, sticky=tk.W, callback=None):
        dropdown_label = customtkinter.CTkLabel(master=frame, text=text, anchor="w")
        dropdown_label.grid(row=row, column=column, padx=padx, pady=pady, sticky=sticky)

        chosen_option = tk.StringVar(value=options[0])

        dropdown_menu = customtkinter.CTkOptionMenu(
            master=frame, values=options, variable=chosen_option, anchor="w",
            command=lambda value: self.update_option(value, callback)
        )
        dropdown_menu.grid(row=row, column=column+1, padx=padx, pady=pady, sticky=sticky)
        dropdown_menu.configure(width=width)

        return dropdown_menu, chosen_option

    def update_option(self, value, callback):
        print(f"Saved option: {value}")
        if callback:
            callback(value)

    def get_images_template(self):
        images_template = xlstools.cell_data_importer(
            openpyxl.load_workbook(self.contraints_excel_filepath, read_only=True)['General'],
            (3,'E'), 
            (21,'E')
        )
        images_template = [item for sublist in images_template for item in sublist if item != 'None']
        return images_template
        


if __name__ == "__main__":
    app = App()
    app.mainloop()

AttributeError: '_tkinter.tkapp' object has no attribute 'images_templates'